In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
%pylab inline
pylab.rcParams['figure.figsize'] = (16, 8)

class linear_regression():
    def __init__(self,poly=1,lmda=0):
        self.poly=poly
        self.lmda=lmda
        self.phi=None
        self.phi_original=None
        self.weights=None
        self.Y=None
        self.rmserr=100.0
    
    #normalization of feature is required to perform ridge regression
    def scale_features(self,mat):
        normed= (mat-mat.mean(axis=0))/mat.std(axis=0)
        #print normed.mean(axis=0),normed.std(axis=0)
        return normed
    
    def transform_feature(self,mat):
        retval=mat
        for fi in xrange(mat.shape[1]):
            for i in xrange(self.poly-1):
                retval=np.hstack((retval,mat[:,fi].reshape((mat.shape[0],1))**(i+2)))
        retval=self.scale_features(retval)
        retval=np.hstack((retval,np.ones(mat.shape[0]).reshape((mat.shape[0],1))))
        return retval
    
    def fit(self,X,Y,algo='ls'):
        self.phi=X.as_matrix()
        self.phi_original=X.as_matrix()
        self.Y=Y.as_matrix()
        self.phi=self.transform_feature(self.phi)
        if algo=='ls':
            self.least_squared_fit()
        
    def least_squared_fit(self):
        #phiT
        phi_t=self.phi.transpose()
        #(phiT*phi)
        temp_l=np.matmul(phi_t,self.phi)
        #lmda*I+phiT*phi
        temp_l=self.lmda*np.identity(temp_l.shape[0])+temp_l
        #inv(phiT*phi)
        phi_inv=np.linalg.inv(temp_l)
        #inv(phiT*phi)*phi
        temp_l_2=np.matmul(phi_inv,phi_t)
        #w=inv(phiT*phi)*phi*Y
        self.weights=np.matmul(temp_l_2,self.Y)
        #row vector to column vector weights
        self.weights=self.weights.reshape((self.weights.size,1))
        #calculate training rmse
        self.rmserr=self.rmse(self.phi_original,self.Y)
        print '<< DONE TRAINING  >>'

    
    def predict(self,x):
        return np.matmul(self.weights.transpose(),self.transform_feature(x).transpose()).transpose()
    
    def rmse(self,x,y):
        err=0.0
        err=((y-self.predict(x))**2).sum()/y.size
        err=err**.5
        return err
        
    def print_weights(self):
        print self.weights.flatten()



SyntaxError: Missing parentheses in call to 'print' (<ipython-input-5-37f7cd66364d>, line 54)

In [6]:
# loading the dataset
data=pd.read_csv('data2.csv')
data.head()
Y=data['h_score']
X=data[data.columns.difference(['h_score'])]

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

l=linear_regression(poly=5,lmda=0.1)
l.fit(X_train,Y_train)

plt.scatter(X_train,Y_train, alpha=0.5,color='blue')
features,predictions=zip(*(sorted(zip(X_train.as_matrix(), l.predict(X_train.as_matrix())))))
plt.plot(features,predictions,'k',color='red')

print l.rmserr
l.print_weights()
plt.show()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-6-4c367109c3d4>, line 18)

In [125]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

X_train=train[['LotArea','OverallQual','OverallCond','YearBuilt','1stFlrSF','GrLivArea']]
Y_train=train[['SalePrice']]
X_test=test[['LotArea','OverallQual','OverallCond','YearBuilt','1stFlrSF','GrLivArea']]
# X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
result=pd.DataFrame(test['Id'])

l2=linear_regression(poly=3,lmda=0.1)
l2.fit(X_train,Y_train)
print l2.rmserr

l2.print_weights()
result['SalePrice']=pd.Series(l2.predict(X_test.as_matrix()).flatten())
# features,predictions=zip(*(sorted(zip(X_train.as_matrix(), list(l2.predict(X_train.as_matrix()))))))
# plt.plot(features,predictions,'k',color='red')

result.to_csv('submission.csv',index=False)


<< DONE TRAINING  >>
30087.4612333
[  25106.74617779   19616.57519664  -19148.00053204  -90804.90198883
  -12680.41040231  -33985.48008662  -39436.48653905   23447.44586842
  -89470.81226413  103679.21808446   77687.00594604  -49028.02662706
    5788.20179925  106893.47467057   59269.5123785   -46763.44360749
  107973.66559392  -54145.20794306  180908.80487638]


In [1]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

def date2t(x):
    l=x.split(':')
    return float(l[-3].split(' ')[-1])+float(l[-2])/60.

def date2m(x):
    l=x.split(" ")
    return int((float(l[0].split('-')[1])+float(l[0].split('-')[2])/30)*2)

train['time']=train['date'].apply(date2t)
train['month']=train['date'].apply(date2m)

test['time']=test['date'].apply(date2t)
test['month']=test['date'].apply(date2m)

train.corr().to_csv('corr.csv')

_train=train[train['Output']<300]

X_train=_train[['time']]
Y_train=_train[['Output']]
X_test=test[['time']]

X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.9, random_state=42)
result=pd.DataFrame(test['Id'])

l2=linear_regression(poly=1,lmda=0.1)
l2.fit(X_train,Y_train)

print "Train Error : ",l2.rmserr
print "Test Error : ",l2.rmse(X_test.as_matrix(),Y_test.as_matrix())

l2.print_weights()
result['Output']=pd.Series(l2.predict(X_test.as_matrix()).flatten())

# features,predictions=zip(*(sorted(zip(X_train.as_matrix(), list(l2.predict(X_train.as_matrix()))))))
# plt.scatter(X_train,Y_train)
# plt.plot(features,predictions,'k',color='red')

result.to_csv('submission.csv',index=False)


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-08f062680fe9>, line 32)